<a href="https://colab.research.google.com/github/chandlersutherland/e12/blob/main/get_rg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GATK requires read group information, so this script is to pull it out of the metadata file provided by Wang Long. The syntax required is 

@RG\tID:{flowcell_ID}.{lane_number}\tPL:ILLUMINA\tPU:{flowcell_ID}.{lane_number}.{library}\tSM:{acession}\tLB:{library} 

Singletons (libraries that were only run once) are processed first, then duplicates (libraries that had to be run twice). This is an artifact of me thinking the duplicates already had read groups, but also makes the file name finding easier since the singletons are named by accession and the duplicates are named by lane info. 

In [1]:
import os 
import pandas as pd


In [43]:
#import file_names document 
metadata = pd.read_excel('/content/drive/MyDrive/Krasileva_Lab/colab_projects/inputs/Col_64_leaves.sequencing_lanes_raw.xlsx')
metadata.rename(columns={'Unnamed: 0':'accession', 'Sample_ID':'library_ID'}, inplace=True)
metadata

#unpack lane info into separate columns 
lane_info = pd.DataFrame()
lane_info[['instrument', 'run_number', 'flowcell_barcode', 'lane_number', 'tile']] = metadata['Lane_info'].str.split(':', expand=True)
metadata = metadata.merge(lane_info, left_index=True, right_index=True).drop(['Lane_info'], axis=1)
metadata

#add accession info based on library id 
file_names = pd.read_excel('/content/drive/MyDrive/Krasileva_Lab/colab_projects/inputs/file names.xlsx')
fn = file_names[['accession', 'library_ID']]
md = fn.merge(metadata, how='inner').sort_values('accession')

##Singletons 

In [88]:
singletons = md[['accession', 'library_ID', 'Lane_ID', 'flowcell_barcode', 'lane_number', 'tile']].drop_duplicates(subset = 'accession', keep = False).reset_index(drop = True)

In [89]:
#test out the syntax 

#@RG\tID:{flowcell_ID}.{lane_number}\tPL:ILLUMINA\tPU:{flowcell_ID}.{lane_number}.{library}\tSM:{accession}\tLB:{library}
accession = singletons.loc[0,'accession']
library = singletons.loc[0,'library_ID']
flowcell_ID = singletons.loc[0,'flowcell_barcode']
lane_number = singletons.loc[0,'lane_number']
print("@RG\\tID:" + str(flowcell_ID) + "." + str(lane_number) + "\\tPL:ILLUMINA\\tPU:" + str(flowcell_ID) + "." + str(lane_number) + "." + str(library) + "\\tSM:" + str(accession) + "\\tLB" + str(library))

@RG\tID:HC2TWBBXX.6\tPL:ILLUMINA\tPU:HC2TWBBXX.6.Col24_SL5-1\tSM:SRR8367186\tLBCol24_SL5-1


In [90]:
#now, write a for loop to output this information into a csv file for adding to samtools files 
filename = "singleton_rg.tsv"
in_dir = "/content/drive/MyDrive/Krasileva_Lab/colab_projects/outputs/"
fh = open(in_dir + filename, 'w')

for i in range(0, len(singletons)):
  accession = singletons.loc[i,'accession']
  library = singletons.loc[i,'library_ID']
  flowcell_ID = singletons.loc[i,'flowcell_barcode']
  lane_number = singletons.loc[i,'lane_number']
  fh.write(str(accession) + '\t' + "'@RG\\tID:" + str(flowcell_ID) + "." + str(lane_number) + "\\tPL:ILLUMINA\\tPU:" + str(flowcell_ID) + "." + str(lane_number) + "." + str(library) + "\\tSM:" + str(accession) + "\\tLB:" + str(library) +"'"+'\n')

fh.close()

In [91]:
#check my work 
newfh = open(in_dir + filename, 'r')
for line in newfh: 
  print(line)

SRR8367186	'@RG\tID:HC2TWBBXX.6\tPL:ILLUMINA\tPU:HC2TWBBXX.6.Col24_SL5-1\tSM:SRR8367186\tLB:Col24_SL5-1'

SRR8367191	'@RG\tID:HC2TWBBXX.1\tPL:ILLUMINA\tPU:HC2TWBBXX.1.Col17_SL4-2\tSM:SRR8367191\tLB:Col17_SL4-2'

SRR8367201	'@RG\tID:HC2TWBBXX.6\tPL:ILLUMINA\tPU:HC2TWBBXX.6.Col24_SL7-2\tSM:SRR8367201\tLB:Col24_SL7-2'

SRR8367203	'@RG\tID:HC2TWBBXX.6\tPL:ILLUMINA\tPU:HC2TWBBXX.6.Col24_SL3-3\tSM:SRR8367203\tLB:Col24_SL3-3'

SRR8367204	'@RG\tID:HC2TWBBXX.6\tPL:ILLUMINA\tPU:HC2TWBBXX.6.Col24_SL4-1\tSM:SRR8367204\tLB:Col24_SL4-1'

SRR8367205	'@RG\tID:HC2TWBBXX.6\tPL:ILLUMINA\tPU:HC2TWBBXX.6.Col24_SL4-2\tSM:SRR8367205\tLB:Col24_SL4-2'

SRR8367206	'@RG\tID:HC2TWBBXX.6\tPL:ILLUMINA\tPU:HC2TWBBXX.6.Col24_SL4-3\tSM:SRR8367206\tLB:Col24_SL4-3'

SRR8367208	'@RG\tID:HC2TWBBXX.6\tPL:ILLUMINA\tPU:HC2TWBBXX.6.Col24_SL5-2\tSM:SRR8367208\tLB:Col24_SL5-2'

SRR8367209	'@RG\tID:HC2TWBBXX.6\tPL:ILLUMINA\tPU:HC2TWBBXX.6.Col24_SL5-3\tSM:SRR8367209\tLB:Col24_SL5-3'

SRR8367210	'@RG\tID:HC2TWBBXX.6\tPL:ILLUMINA\t

##Read group Files


In [92]:
#first, have to tidy the filenames file by putting the filenames in the same column

#read in file names document, subset to accession, library ID, and filenames 
fn2 = file_names[['accession', 'library_ID', 'filename', 'filename2', 'filename3', 'filename4']]

#make a dataframe of only the libraries that were run twice by grabbing filename 3 and 4, dropping nas, and making this filename and filename2 
double_runs = fn2[['accession', 'library_ID', 'filename3', 'filename4']].dropna().rename(columns = {'filename3':'filename', 'filename4':'filename2'})
double_runs

#grab the single runs/first run for double 
single_runs = fn2[['accession', 'library_ID', 'filename', 'filename2']]
single_runs

#merge together, creating two rows per accession if the library had to be run twice. Change filename format as appropriate 
together = double_runs.append(single_runs).sort_values('accession')[['accession', 'library_ID', 'filename']]
together['filename'] = together['filename'].str.replace('_1.fq.gz', '.bam')

#add a flowcell barcode so I can merge with the original metadata file 
together['flowcell_barcode'] = together['filename'].str.lstrip("FC").str[:-23].str.rstrip("_")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  app.launch_new_instance()


In [93]:
#merge together to create a dataframe with accession, all the RG info, and the filename
all_names = md.merge(together, on=['accession', 'library_ID', 'flowcell_barcode'])

#filter by duplicate values, since all the singletons already have rgs added 
duplicate_names = all_names[-all_names.accession.isin(singletons.accession)].reset_index(drop = True) 
duplicate_names

,accession,library_ID,DNA_ID,Lane_ID,Barcode,Read_Pairs,GC%,Raw_Data(G),Raw_Depth,instrument,run_number,flowcell_barcode,lane_number,tile,filename
0,SRR8367185,Col24_RL2-1,24_21,Lane6,TAATGTTG,11861549,41.0,3.558465,29.653872,K00137,295,HC2V5BBXX,2,1101,FCHC2V5BBXX_L2_wHAXPI032499-26.bam
1,SRR8367185,Col24_RL2-1,24_21,Lane4,TAATGTTG,10841218,41.0,3.252365,27.103045,K00137,285,H7NHMBBXX,1,1101,FCH7NHMBBXX_L1_wHAXPI032499-26.bam
2,SRR8367187,Col24_RL2-3,24_23,Lane6,GTTACTTG,13635592,39.0,4.090678,34.088980,K00137,295,HC2V5BBXX,2,1101,FCHC2V5BBXX_L2_wHAXPI032500-27.bam
3,SRR8367187,Col24_RL2-3,24_23,Lane4,GTTACTTG,10731726,38.0,3.219518,26.829315,K00137,285,H7NHMBBXX,1,1101,FCH7NHMBBXX_L1_wHAXPI032500-27.bam
4,SRR8367188,Col24_RL4-2,24_42,Lane4,CTTGATGG,13980544,40.0,4.194163,34.951360,K00137,285,H7NHMBBXX,1,1103,FCH7NHMBBXX_L1_wHAXPI032505-32.bam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,SRR8367245,Col24_RL5-3,24_53,Lane4,TATTCGCG,11921918,41.0,3.576575,29.804795,K00137,285,H7NHMBBXX,1,1104,FCH7NHMBBXX_L1_wHAXPI032508-36.bam
60,SRR8367246,Col24_RL5-1,24_51,Lane4,CTAAGTCG,12077593,40.0,3.623278,30.193983,K00137,285,H7NHMBBXX,1,1104,FCH7NHMBBXX_L1_wHAXPI032507-35.bam
61,SRR8367246,Col24_RL5-1,24_51,Lane6,CTAAGTCG,15110008,40.0,4.533002,37.775020,K00137,295,HC2V5BBXX,2,1103,FCHC2V5BBXX_L2_wHAXPI032507-35.bam
62,SRR8367248,Col17_RL4-1,17_41,Lane6,CTAGTTAT,8056094,39.0,2.416828,20.140235,K00137,295,HC2V5BBXX,2,1106,FCHC2V5BBXX_L2_wHAXPI032445-20.bam


In [95]:
#write accession, filename, and rg info to duplicate_names.tsv

filename = "duplicate_names.tsv"
in_dir = "/content/drive/MyDrive/Krasileva_Lab/colab_projects/outputs/"
fh = open(in_dir + filename, 'w')

for i in range(0, len(duplicate_names)):
  accession = duplicate_names.iloc[i,0]
  library = duplicate_names.loc[i,'library_ID']
  flowcell_ID = duplicate_names.loc[i,'flowcell_barcode']
  lane_number = duplicate_names.loc[i,'lane_number']
  fn = duplicate_names.loc[i, 'filename']
  fh.write(str(accession) + '\t' + 
           str(fn) + '\t' +
           "'@RG\\tID:" + str(flowcell_ID) + "." + str(lane_number) + 
           "\\tPL:ILLUMINA\\tPU:" + str(flowcell_ID) + "." + str(lane_number) + "." + str(library) + 
           "\\tSM:" + str(accession) + "\\tLB:" + str(library) +"'"+'\n')
fh.close()

newfh = open(in_dir + filename, 'r')
for line in newfh: 
  print(line)

SRR8367185	FCHC2V5BBXX_L2_wHAXPI032499-26.bam	'@RG\tID:HC2V5BBXX.2\tPL:ILLUMINA\tPU:HC2V5BBXX.2.Col24_RL2-1\tSM:SRR8367185\tLB:Col24_RL2-1'

SRR8367185	FCH7NHMBBXX_L1_wHAXPI032499-26.bam	'@RG\tID:H7NHMBBXX.1\tPL:ILLUMINA\tPU:H7NHMBBXX.1.Col24_RL2-1\tSM:SRR8367185\tLB:Col24_RL2-1'

SRR8367187	FCHC2V5BBXX_L2_wHAXPI032500-27.bam	'@RG\tID:HC2V5BBXX.2\tPL:ILLUMINA\tPU:HC2V5BBXX.2.Col24_RL2-3\tSM:SRR8367187\tLB:Col24_RL2-3'

SRR8367187	FCH7NHMBBXX_L1_wHAXPI032500-27.bam	'@RG\tID:H7NHMBBXX.1\tPL:ILLUMINA\tPU:H7NHMBBXX.1.Col24_RL2-3\tSM:SRR8367187\tLB:Col24_RL2-3'

SRR8367188	FCH7NHMBBXX_L1_wHAXPI032505-32.bam	'@RG\tID:H7NHMBBXX.1\tPL:ILLUMINA\tPU:H7NHMBBXX.1.Col24_RL4-2\tSM:SRR8367188\tLB:Col24_RL4-2'

SRR8367188	FCHC2V5BBXX_L2_wHAXPI032505-32.bam	'@RG\tID:HC2V5BBXX.2\tPL:ILLUMINA\tPU:HC2V5BBXX.2.Col24_RL4-2\tSM:SRR8367188\tLB:Col24_RL4-2'

SRR8367189	FCH7NHMBBXX_L1_wHAXPI032506-34.bam	'@RG\tID:H7NHMBBXX.1\tPL:ILLUMINA\tPU:H7NHMBBXX.1.Col24_RL4-3\tSM:SRR8367189\tLB:Col24_RL4-3'

SRR8367189	FC